In [ ]:
import tensorflow as tf
import os

from tensorflow.keras import layers, models
import cv2
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
real_dir="/content/drive/MyDrive/MSU-MFSD/pics/attack"
attack_dir="/content/drive/MyDrive/MSU-MFSD/pics/attack"

In [ ]:
for i in os.listdir(real_dir):
  img=cv2.imread(os.path.join(real_dir,i))
  img=cv2.resize(img,(64,64))
  img=img/255.0

In [ ]:
for i in os.listdir(attack_dir):
  img=cv2.imread(os.path.join(attack_dir,i))
  img=cv2.resize(img,(64,64))
  img=img/255.0
print(img.shape)

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
height,width=64,64
x = []
y = []

for f in os.listdir(real_dir):
    if f.lower().endswith((".jpg", ".png")):
        img = cv2.imread(os.path.join(real_dir, f))
        if img is not None:
            img = cv2.resize(img, (height, width))  # resize if needed
            x.append(img)
            y.append(1)  # label


In [ ]:
for g in os.listdir(attack_dir):
  if g.lower().endswith((".jpg",".png")):
    img=cv2.imread(os.path.join(attack_dir,g))
    if img is not None:
      img=cv2.resize(img,(height,width))
      x.append(img)
      y.append(0)

In [ ]:
from sklearn.model_selection import train_test_split
x = np.array(x, dtype=np.float32) / 255.0
y = np.array(y, dtype=np.int32)


In [ ]:
xtr,xte,ytr,yte=train_test_split(x,y,test_size=0.2,random_state=42)
xtr=np.array(xtr,dtype=np.float32)
ytr=np.array(ytr,dtype=np.float32)


In [ ]:
history = model.fit(xtr,ytr,epochs=50,batch_size=32,validation_split=0.2,shuffle=True)


In [ ]:
ypreds=model.predict(xte)
print(ypreds)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(yte,ypreds.round()))
print(precision_score(yte,ypreds.round()))
print(recall_score(yte,ypreds.round()))
print(f1_score(yte,ypreds.round()))